In [48]:
import pandas as pd
import os
#import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.dates import MonthLocator, DateFormatter


from sqlalchemy import create_engine, text

#maximum number of rows to display
pd.options.display.max_rows = 50

DB_USERNAME = 'developers'
DB_PASSWORD = '12345678'
DB_INSTANCE = 'database-1.cmostnjfvkd6.us-east-2.rds.amazonaws.com'

#DB_INSTANCE = 'localhost'
DB_PORT = '5432'
DB_DATA_BASE = 'cnc'

engine=create_engine(f'postgresql://{DB_USERNAME}:{DB_PASSWORD}@{DB_INSTANCE}:{DB_PORT}/{DB_DATA_BASE}')

def runQuery(sql):
    result = engine.connect().execution_options(isolation_level="AUTOCOMMIT").execute((text(sql)))
    return pd.DataFrame(result.fetchall(), columns=result.keys())


query_03= """
select distinct tabla1.* from(
select tabla.*, cu.etiquetas_rango_edad from(
select codenc, telelink, fecha, duracion, edad,  "ciudad", "estrato", edcc.etiqueta as "genero", cod_rango_edad from (
select codenc, telelink, fecha, duracion, edad, "cod_ciudad", "ciudad", estrato as "cod_estrato", 
edcc.etiqueta as "estrato", genero, cod_rango_edad  from(
select codenc, telelink, fecha, duracion, edad, ciudad as "cod_ciudad", edcc.etiqueta as "ciudad", estrato, genero, cod_rango_edad
from ecar_dwh.ecar_dwh_base_encuestas_efectivas edbee 
inner join ecar_dwh.ecar_dwh_base_correspondencia_codigos edcc on edbee.ciudad=edcc.codigo
where edcc.campo='ciudad')tt
inner join ecar_dwh.ecar_dwh_base_correspondencia_codigos edcc on tt.estrato=edcc.codigo
where edcc.campo='estrato')tt1
inner join ecar_dwh.ecar_dwh_base_correspondencia_codigos edcc on tt1.genero=edcc.codigo
where edcc.campo='genero')tabla
left join ecar_dwh.ecar_dwh_base_muestra_ecar_mensual_cuotas cu on cu.cod_rango_edad=tabla.cod_rango_edad)tabla1
"""
df_efectivas = runQuery(query_03)
df_efectivas['etiquetas_rango_edad'].replace('12 a 17 años','12 a 17',inplace=True)

In [49]:
df_efectivas_cuotas=pd.DataFrame()
for i in ['estrato', 'genero','etiquetas_rango_edad']:
    df_count=df_efectivas.groupby(i)[['codenc']].count().rename(columns={'codenc':"count"})
    df_count['variable']=i
    df_efectivas_cuotas=pd.concat([df_count, df_efectivas_cuotas])

In [50]:
df_efectivas_cuotas=df_efectivas_cuotas.sort_values('count',ascending=True)

In [66]:
df_efectivas[(df_efectivas['codenc']=='Encuestador 137') | (df_efectivas['codenc']=='Encuestador 10')]

,codenc,telelink,fecha,duracion,edad,ciudad,estrato,genero,cod_rango_edad,etiquetas_rango_edad
77,Encuestador 10,210910.0,2018-04-29,1025,46,Medellín,Bajo,Hombre,5,45 a 54
78,Encuestador 10,210942.0,2018-04-29,1109,49,Medellín,Medio Bajo,Hombre,5,45 a 54
79,Encuestador 10,211098.0,2018-04-30,916,19,Medellín,Bajo,Mujer,2,18 a 24
80,Encuestador 10,211149.0,2018-04-30,492,55,Cartagena,Bajo,Mujer,1,12 a 17
81,Encuestador 10,NaN,2018-05-04,638,57,Cali,Bajo,Hombre,1,12 a 17
...,...,...,...,...,...,...,...,...,...,...
18026,Encuestador 137,NaN,2018-12-17,1080,17,Bogotá,Medio Bajo,Mujer,1,12 a 17
18027,Encuestador 137,NaN,2018-12-18,684,67,Bogotá,Bajo,Mujer,1,12 a 17
18028,Encuestador 137,NaN,2018-12-18,815,16,Bogotá,Bajo,Hombre,1,12 a 17
18029,Encuestador 137,NaN,2018-12-18,945,15,Bogotá,Bajo,Mujer,1,12 a 17


In [52]:

query_02=""" select * from ecar_dwh.ecar_dwh_base_modelo """
df_modelo=runQuery(query_02)

cols=['calls', 'total', 'espera', 'hablado', 'disponible', 'pausas',
       'muerto', 'duracion_efectivas','efectividad_ajustada']

df_tiempos=df_modelo[cols]

In [53]:
df_tiempos

,calls,total,espera,hablado,disponible,pausas,muerto,duracion_efectivas,efectividad_ajustada
0,6252.705882,623484.000000,150572.000000,313690.000000,42598.000000,116624.000000,2193.000000,1102.323944,1804.235294
1,10989.000000,147804.000000,40859.500000,61113.500000,14951.500000,30879.500000,289.000000,954.035714,1188.000000
2,6900.927106,820764.181818,235728.090909,330865.454545,88244.272727,165926.363636,7262.818182,739.523805,2980.843057
3,4202.400000,125574.500000,37047.000000,49981.500000,9136.000000,29410.000000,3586.500000,836.228571,2268.000000
4,6440.057143,510847.500000,180869.500000,202350.000000,30753.500000,96874.500000,5939.500000,749.815857,1627.714286
...,...,...,...,...,...,...,...,...,...
279,7177.520000,760716.000000,226663.000000,311614.333333,82926.000000,139512.666667,4070.333333,719.768171,3102.240000
280,6085.707762,501703.800000,156219.600000,174043.800000,79735.400000,91705.000000,5142.000000,818.476296,2383.644187
281,9660.391111,554952.000000,112426.666667,252206.666667,69592.000000,120726.666667,4709.000000,1077.662688,2026.880000
282,4788.000000,604918.000000,134926.000000,305491.000000,40198.000000,124303.000000,2648.000000,993.446809,2256.000000


In [54]:
df_tiempos[df_tiempos['efectividad_ajustada'].isnull()==True]

,calls,total,espera,hablado,disponible,pausas,muerto,duracion_efectivas,efectividad_ajustada
198,NaN,32001.0,11897.0,9051.0,4327.0,6726.0,8.0,NaN,NaN


In [55]:
cols=['efectivas_mujer', 'efectivas_hombre','efectivas_alto', 'efectivas_medio_alto', 'efectivas_medio_medio','efectivas_bajo', 'efectivas_medio_bajo', 'efectivas_tunja',
       'efectivas_popayán', 'efectivas_montería', 'efectivas_manizales','efectivas_armenia', 'efectivas_villavicencio', 'efectivas_pasto','efectivas_cúcuta', 'efectivas_santa_marta', 'efectivas_ibagué',
       'efectivas_neiva', 'efectivas_pereira', 'efectivas_cartagena','efectivas_cali', 'efectivas_bucaramanga', 'efectivas_barranquilla','efectivas_medellín', 'efectivas_bogotá', 'efectivas_rango_edad_5',
       'efectivas_rango_edad_2', 'efectivas_rango_edad_4','efectivas_rango_edad_3', 'efectivas_rango_edad_1']
df_modelo=df_modelo[df_modelo['codenc']!='Encuestador 290']
for i in cols:
    df_modelo["%" + str(i)]=round((df_modelo[i]/df_modelo['efectividad_ajustada'])*100).astype(int)
df_porc_tabla=df_modelo[['codenc','efectividad_ajustada','meses_trabajados','como_conocio_cnc','localidad','educacion_formal','estado_educacion','nombre_educacion',
         '%efectivas_mujer', '%efectivas_hombre', '%efectivas_alto','%efectivas_medio_alto', '%efectivas_medio_medio',
         '%efectivas_bajo', '%efectivas_medio_bajo','%efectivas_tunja', '%efectivas_popayán','%efectivas_montería', 
         '%efectivas_manizales','%efectivas_armenia', '%efectivas_villavicencio','%efectivas_pasto', '%efectivas_cúcuta',
         '%efectivas_santa_marta', '%efectivas_ibagué','%efectivas_neiva', '%efectivas_pereira','%efectivas_cartagena', '%efectivas_cali',
         '%efectivas_bucaramanga', '%efectivas_barranquilla','%efectivas_medellín', '%efectivas_bogotá','%efectivas_rango_edad_5', '%efectivas_rango_edad_2',
         '%efectivas_rango_edad_4', '%efectivas_rango_edad_3','%efectivas_rango_edad_1']]

In [56]:
pd.options.display.max_rows =20
df_modelo.columns

Index(['codenc', 'calls', 'total', 'espera', 'hablado', 'disponible', 'pausas',
       'muerto', 'duracion_efectivas', 'dias_trabajados',
       'efectividad_ajustada', 'efectivas_mujer', 'efectivas_hombre',
       'efectivas_alto', 'efectivas_medio_alto', 'efectivas_medio_medio',
       'efectivas_bajo', 'efectivas_medio_bajo', 'efectivas_tunja',
       'efectivas_popayán', 'efectivas_montería', 'efectivas_manizales',
       'efectivas_armenia', 'efectivas_villavicencio', 'efectivas_pasto',
       'efectivas_cúcuta', 'efectivas_santa_marta', 'efectivas_ibagué',
       'efectivas_neiva', 'efectivas_pereira', 'efectivas_cartagena',
       'efectivas_cali', 'efectivas_bucaramanga', 'efectivas_barranquilla',
       'efectivas_medellín', 'efectivas_bogotá', 'efectivas_rango_edad_5',
       'efectivas_rango_edad_2', 'efectivas_rango_edad_4',
       'efectivas_rango_edad_3', 'efectivas_rango_edad_1', 'meses_trabajados',
       'competencias_funcionales', 'numeros', 'lectura_voz_alta',
      

In [64]:
df_encuestadores_efec=df_porc_tabla[['codenc','efectividad_ajustada','%efectivas_hombre','%efectivas_mujer']].sort_values('efectividad_ajustada',ascending=False)
df_encuestadores_efec['suma']=df_encuestadores_efec['%efectivas_hombre']+df_encuestadores_efec['%efectivas_mujer']
df_encuestadores_efec[df_encuestadores_efec['suma']!=100]

,codenc,efectividad_ajustada,%efectivas_hombre,%efectivas_mujer,suma
177,Encuestador 270,3703.006175,41,52,93
189,Encuestador 282,3355.934452,47,44,91
72,Encuestador 169,3291.822163,48,46,94
108,Encuestador 204,3263.709849,43,38,81
145,Encuestador 240,3182.787692,37,46,83
...,...,...,...,...,...
1,Encuestador 10,1188.000000,18,18,36
40,Encuestador 137,987.428571,0,0,0
33,Encuestador 130,911.153110,35,44,79
220,Encuestador 311,864.000000,0,0,0
